### Explanation of steps to generate the syntelog files
1: drop one of the duplicated lines <br><br>
2: drop if excess in ATL1-4 <br><br>
3: get rid of 0 and/or S if there are enough without it; if there are enough with 0 but without S, drop S.<br><br>
4: drop mixed ATL chromosomes, keep if S, see next step for "problem0"<br><br>
5: for problem 0s, drop the 0 if it's in the minority (i.e., chr5hap1, chr5hap2, chr5hap0, chr8hap0, drop chr8hap8 and leave the rest).  If they are equal counts still, just drop the row entirely<br><br>
6: manually curate any remaining problem genes (3 genes at this point)<br><br>
7: remove ones with duplications in DM and at least 1 wild, since these would be likely to be accidental merging of two genes<br><br>
8.1: drop if doesn't contain ATL alleles <---- syntelogs_atl_only.csv<br><br>
8.2: drop if excess in CND or M6 (based on step 7, not step 8.1) <---- syntelogs_3species.csv

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
filename = "/Users/fekeann/Documents/Lab/RNAseq/Syntelogs/circadian_atl_comb_syntelogs.txt"

In [3]:
# read in the file
with open(filename) as f:
    lines = f.read().splitlines()

In [4]:
# turn each line into a nested list
# and make another version that is a flat list one geneID deep (for Pandas, later)
split = [line.split('\t') for line in lines]
flat_split = [el for line in split for el in line]

In [5]:
# make a new list of "SyntelogIDs" that contains the right number for each row
split_groups = []
for i, line in enumerate(split):
    for j in range(len(line)):
        split_groups.append(f"Synt_{i}")

In [6]:
# turn the flat list and the SyntelogIDs into a pandas data frame
syntelogs = pd.DataFrame({"Syntelog":split_groups, "geneID":flat_split})

In [7]:
# find the species in the geneID
# if it's S. tubersum, the "species" is actually the variety (ATL or DM)
def determine_species(gene):
    if "Soltu" in gene:
        return gene.split(".")[1].split("_")[0]
    else:
        return gene.split(".")[0]

In [8]:
# find which chromosome (*not* haplotype) the gene is on
# since Soltu and Solch contain variety information while Solca and Sollyc don't
# then it is in a different spot dependent on the species
def determine_chrom(gene):
    if "Soltu" in gene or "Solch" in gene:
        gene = gene.split(".")[2]
        if "G" in gene:
            chrom = gene.split("G")[0].split("_")[0]
        else:
            chrom = gene[0]
    elif "Sollyc" in gene or "Solca" in gene:
        gene = gene.split(".")[1]
        if "G" in gene:
            chrom = gene.split("G")[0]
        else:
            chrom = gene[0]
    try:
        return int(chrom)
    except ValueError:
        return(chrom)

In [9]:
# add new columns to the data frame containing this info
syntelogs["species"] = syntelogs.geneID.apply(determine_species)
syntelogs["chromosome"] = syntelogs.geneID.apply(determine_chrom)

In [10]:
def reg_find(regex, text):
    if re.search(regex, text):
        return True
    else:
        return False

In [11]:
def find_atl_hap(geneID):
    # finds the haplotype for ATL gene IDs
    # would provide meaningless results for other varieties
    if ".S" in geneID:
        return "S"
    else:
        ID = geneID.split("_")[2]
        return ID[:2]

In [12]:
hap_list = ["DM", "Solch", "Solca", "Sollyc", "_0G", "_1G", "_2G", "_3G", "_4G"]
def quality_scorer(dataset):
    excesscount_dict = {}
    missingcount_dict = {}
    atlcount_dict = {}
    excessid_dict = {}
    missingid_dict = {}
    atlissues = {}
    all_synts = dataset.Syntelog.unique()
    total_synts = len(all_synts)
    cur_synt = 0 #this is just for progress displays
    for synt in all_synts:
        cur_synt += 1
        atl_doubles = False
        print(f"On Syntelog {cur_synt} of {total_synts}", end='\r')
        cur_syntelog = dataset[dataset.Syntelog == synt] #filter to the one we're looking at
        for hap in hap_list:
            cur_count = sum(cur_syntelog.geneID.str.contains(hap)) # count how many there are in each haplotype
            if cur_count > 1: # if there is more than one in that haplotype
                excesscount_dict[hap] = excesscount_dict.get(hap, 0) + 1 #record that we found another
                temp = excessid_dict.get(hap, [])
                temp.append(synt)
                excessid_dict[hap] = temp # and record what haplotype had the issue
                if reg_find("_[1-4]G", hap): # if the issue was in one of the phased ATL haplotypes, note it
                    atl_doubles = True
            elif cur_count == 0:
                missingcount_dict[hap] = missingcount_dict.get(hap, 0) + 1 # record if something isn't there too
                temp = missingid_dict.get(hap, [])
                temp.append(synt)
                missingid_dict[hap] = temp
        cur_atl = cur_syntelog[cur_syntelog.species == "Atl"].copy() # now just focus on ATL
        atl_count = cur_atl.shape[0]
        atlcount_dict[atl_count] = atlcount_dict.get(atl_count, 0) + 1
        cur_atl["Haplotype"] = cur_atl.geneID.apply(find_atl_hap) # make a new column containing the haplotype 
        if atl_count > 4:
            if atl_doubles == False: # if the problem is in the 0 or S haplotypes, think further
                                     # if the problem is in a phased haplotype, we will need to discard it later.
                cur0s = cur_atl[cur_atl.Haplotype=="0G"].shape[0]
                curSs =  cur_atl[cur_atl.Haplotype=="S"].shape[0]
                if atl_count - 4 == curSs: # if the entire problem is extra alleles from the scaffold, record it for removal
                    temp = atlissues.get("RemoveSs", [])
                    temp.append(synt)
                    atlissues["RemoveSs"] = temp
                elif atl_count - 4 < curSs: # if the problem is extra alleles from the scaffold but non-1:1 in the phased
                    temp = atlissues.get("ManuallyCurateS", [])  # mark it for manual curation
                    temp.append(synt)
                    atlissues["ManuallyCurateS"] = temp
                elif atl_count - 4 == cur0s: # if the entire problem is extra alleles from the unphased haplotype, record it for removal
                    temp = atlissues.get("Remove0s", [])
                    temp.append(synt)
                    atlissues["Remove0s"] = temp
                elif atl_count - 4 < cur0s: # if the problem is extra alleles from the unphased but non-1:1 in the phased, proceed.
                    if len(cur_atl.chromosome.unique()) > 1: # if the unphased are on multiple chromosomes, mark
                        temp = atlissues.get("Mixed0Chrom", [])
                        temp.append(synt)
                        atlissues["Mixed0Chrom"] = temp
                    else:
                        temp = atlissues.get("ManuallyCurate0", []) # if there are just too many unphased, mark
                        temp.append(synt)
                        atlissues["ManuallyCurate0"] = temp
                elif atl_count - 4 == curSs + cur0s: # if it's perfect outside of unphased and scaffold
                    temp = atlissues.get("Remove0andS", []) # mark for removal
                    temp.append(synt)
                    atlissues["Remove0andS"] = temp
                else:
                    temp = atlissues.get("Escaped", []) #a catch-all for anything that didn't fall into the above categories
                    temp.append(synt)
                    atlissues["Escaped"] = temp
    print("")
    print("Excess Counts:")
    print(excesscount_dict)
    print("Missing Counts:")
    print(missingcount_dict)
    print("ATL Counts:")
    print(atlcount_dict)
    return excessid_dict, missingid_dict, atlissues,

In [13]:
init_excess, init_missing, init_atl = quality_scorer(syntelogs)
# since we don't care if there is at least one per species, then init_missing is unused\

On Syntelog 30457 of 30457
Excess Counts:
{'_2G': 93, 'Solch': 119, 'Solca': 119, '_3G': 207, '_4G': 64, 'Sollyc': 69, '_0G': 371, 'DM': 137, '_1G': 135}
Missing Counts:
{'Solch': 7940, 'Sollyc': 9916, '_0G': 23149, '_1G': 14253, '_2G': 12634, '_3G': 12182, '_4G': 13815, 'Solca': 7189}
ATL Counts:
{0: 5695, 4: 11725, 1: 2828, 2: 3975, 3: 5797, 5: 384, 7: 12, 6: 32, 8: 6, 9: 2, 12: 1}


In [14]:
# there were a handful of rows in the initial file that were duplicated
duped_rows = syntelogs["geneID"].value_counts()>1
duped_synts = syntelogs[syntelogs.geneID.isin(duped_rows[duped_rows==True].index)].copy()

In [15]:
dupe_pairs = dict()
double_dippers = []
for gene in duped_synts.cur_synts = duped_synts[duped_synts.geneID==gene].Syntelog.to_list()
    for i in range(2): #check both orientations
        if cur_synts[i] in dupe_pairs.keys(): # if one is already in the keys of dupe_pairs
            found = True
            if cur_synts[abs(i-1)] == dupe_pairs[cur_synts[i]]: # make sure that the other pair is there
                pass # if it's there, then we're good, can do nothing
            else: double_dippers.append(cur_synts[i]) #otherwise record it as occuring with multiple pairings
    if found == False:
        dupe_pairs[cur_synts[0]] = cur_synts[1] #if we haven't seen either before, add it to the list of pairs

# if the number of "double dippers" is 0, that means entire lines we're duplicated and we're free to just toss one.
print(len(double_dippers))geneID.unique():
    # go through all the duplicated ones
    found = False
    # find out which pairs are duplicates of eachother
    

0


In [16]:
# since the output of the previous cell is 0, can just keep one at random and toss the other
# arbitrarily, we decide to keep the first (the key) and toss the second (the value)
step1 = syntelogs[~syntelogs.Syntelog.isin(dupe_pairs.values())].copy()

In [17]:
keys_to_drop = ["_1G", "_2G", "_3G", "_4G"]
step2 = step1.copy()
# if there is a problem in the ATL phased haplotypes, get rid of that syntelog.
for key in keys_to_drop:
    step2 = step2[~step2.Syntelog.isin(init_excess[key])]

In [18]:
def drop0G(syntelog, df):
    """
    Gets rid of the unphased haplotype genes for a specific syntelog 
    syntelog: a str containing the name of the syntelog to be altered
    df: the pandas dataframe to be alteredd
    """
    cur_synt  = df[df.Syntelog == syntelog]
    df.drop(cur_synt[cur_synt.geneID.str.contains("_0G")].index, inplace=True)

In [19]:
def dropS(syntelog, df):
    """
    Gets rid of the unphased scaffold genes for a specific syntelog 
    syntelog: a str containing the name of the syntelog to be altered
    df: the pandas dataframe to be alteredd
    """
    cur_synt = df[df.Syntelog == syntelog]
    cur_atl = cur_synt[cur_synt.species == "Atl"]
    df.drop(cur_atl[cur_atl.chromosome == "S"].index, inplace=True)

In [20]:
step3 = step2.copy()
# remove the 0 and S Atlantic genes as appropriate, judged by "quality_scorer" above
remove0 = init_atl["Remove0s"] + init_atl["Remove0andS"]
removeS = init_atl["RemoveSs"] + init_atl["Remove0andS"]
[drop0G(synt, step3) for synt in remove0];
[dropS(synt, step3) for synt in removeS];

In [21]:
mixed_chrom_synts = []
mixed_chrom_on0 = []
num_synt = step3.Syntelog.unique().shape[0]
for i, synt in enumerate(step3.Syntelog.unique()):
    print(f"On Syntelog {i+1} of {num_synt}", end='\r')
    cur_synt = step3[step3.Syntelog==synt]
    cur_atl = cur_synt[cur_synt.species=="Atl"]
    # look for mixed chromosome ATL synts
    if cur_atl.chromosome.unique().shape[0] > 1:
        cur_atl = cur_atl.copy()
        # if it's mixed, try dropping the scaffold; if it's fixed, it's fine to keep
        dropS(synt, cur_atl)
        if cur_atl.chromosome.unique().shape[0] > 1:
            drop0G(synt, cur_atl) # now try dropping the unphased
            if cur_atl.chromosome.unique().shape[0] > 1:
                mixed_chrom_synts.append(synt) # if that doesn't fix it, mark it one way
            else:
                mixed_chrom_on0.append(synt) # if the problem *is* the unphased, mark that instead

On Syntelog 30018 of 30018

In [22]:
# if it's actually mixed (not caused by the scaffold or the unphased, get rid of it)
step4 = step3[~step3.Syntelog.isin(mixed_chrom_synts)].copy() 

In [23]:
mixed_chrom_still = []
num_synt = len(mixed_chrom_on0)
for i, synt in enumerate(mixed_chrom_on0): # this next bit only applies to the rows where the problem is on the unphased
    print(f"On Syntelog {i+1} of {num_synt}", end='\r')
    cur_synt = step4[step4.Syntelog==synt]
    cur_atl = cur_synt[cur_synt.species=="Atl"].copy()
    dropS(synt, cur_atl)
    if cur_atl.chromosome.value_counts(ascending=False).values[0] > 1: # if there is more than one chromosome after removing the S
        keep = cur_atl.chromosome.value_counts(ascending=False).index[0] # keep the versions that belong to the majority chromosome
        step4.drop(cur_atl[cur_atl.chromosome != keep].index, inplace=True)
    else:
        mixed_chrom_still.append(synt) # if there isn't a majority, mark it

On Syntelog 92 of 92

In [24]:
step5 = step4[~step4.Syntelog.isin(mixed_chrom_still)].copy() # keep those that aren't still problematic for mixed chromosomes

In [25]:
mod_excess, mod_missing, mod_atl = quality_scorer(step5) #figure out which still need to be manually filtered

On Syntelog 29457 of 29457
Excess Counts:
{'Sollyc': 47, '_0G': 320, 'DM': 80, 'Solch': 75, 'Solca': 73}
Missing Counts:
{'Solch': 7858, 'Sollyc': 9763, '_0G': 22740, '_1G': 13973, '_2G': 12410, '_3G': 11882, '_4G': 13408, 'Solca': 7108}
ATL Counts:
{0: 5695, 4: 11615, 1: 2828, 2: 3877, 3: 5439, 5: 3}


In [26]:
mod_atl

{'ManuallyCurateS': ['Synt_26033', 'Synt_26286'],
 'ManuallyCurate0': ['Synt_27046']}

In [27]:
step5[step5.Syntelog=="Synt_27046"]
#BLAST Searching using the 2G005970.2 CDS as the query returns 0G007690.1 as hit #7
#Hits 1-6 belong to different spiceoforms of the 2G, 3G, and 4G versions on this table
#0G010330 doesn't even show up in the top 50 hits, so drop this one.
to_drop = ["Soltu.Atl_v3.11_0G010330.1"]

In [28]:
step5[step5.Syntelog=="Synt_26033"]
#BLAST Searching using the 10_2G014510.1 CDS as the query returns S059640.1 as hit #1
#S059540.1 is hit #3.  The scores (e.value, ID, and query coverage) are all equal.
#There are also versions of the one on 1, and 2 nearby too (it's likely a real duplication)
#Since the 1 and 2 versions picked the earlier (lower) # as the syntelog,
#I will do the same and pick S059540 as the version to keep
to_drop.append("Soltu.Atl_v3.S059640.1")

In [29]:
step5[step5.Syntelog=="Synt_26286"]
#BLAST searching using the 11_1G001150.1 CDS as the query returns S006220 as 
#the 19th hit and S017940 as the 24th hit.  
#S006220 has considerably lower e-value (1e-116 vs 4e-55), better coverage (100 vs 56.91)
#and better TopID (96.76 vs 95.04)
#For this reason keep S006220.
to_drop.append("Soltu.Atl_v3.S017940.1")

In [30]:
step6 = step5[~step5.geneID.isin(to_drop)]
step6 = step6.copy()

In [31]:
step6_excess, _, step6_atl = quality_scorer(step6)
step6_atl # just double check that the ATL is all good now

On Syntelog 29457 of 29457
Excess Counts:
{'Sollyc': 47, '_0G': 319, 'DM': 80, 'Solch': 75, 'Solca': 73}
Missing Counts:
{'Solch': 7858, 'Sollyc': 9763, '_0G': 22740, '_1G': 13973, '_2G': 12410, '_3G': 11882, '_4G': 13408, 'Solca': 7108}
ATL Counts:
{0: 5695, 4: 11618, 1: 2828, 2: 3877, 3: 5439}


{}

In [32]:
# if there is 2 in DM and 2 in either wild, than that feels like it could actually be 2 genes
# that were mis-identified as syntelogs; so we should filter those out as well
probable_merge = []
for gene in step6_excess["DM"]:
    if gene in step6_excess["Solch"] or gene in step6_excess["Solca"]:
        probable_merge.append(gene)

In [94]:
step7 = step6[~step6.Syntelog.isin(probable_merge)].copy()
step7_excess, _, _ = quality_scorer(step7)

On Syntelog 29433 of 29433
Excess Counts:
{'Sollyc': 40, '_0G': 318, 'DM': 56, 'Solch': 54, 'Solca': 60}
Missing Counts:
{'Solch': 7858, 'Sollyc': 9755, '_0G': 22721, '_1G': 13962, '_2G': 12401, '_3G': 11873, '_4G': 13397, 'Solca': 7103}
ATL Counts:
{0: 5691, 4: 11608, 1: 2827, 2: 3871, 3: 5436}


In [95]:
# if there is no ATL copy, get rid of that syntelog since it's not useful for the haplotype paper
step8_1 = step7.copy()
step8_1 = step8_1[step8_1.Syntelog.isin(step8_1[step8_1.species=="Atl"].Syntelog)]
# we also only want ATL and DM in this final file, both of which are 'Soltu')
step8_1 = step8_1[step8_1.geneID.str.contains("Soltu")]

In [96]:
# for the other paper, we don't want any problem syntelogs in Solca or Solch
keys_to_drop = ["Solca", "Solch"]
step8_2 = step7.copy()
for key in keys_to_drop:
    step8_2 = step8_2[~step8_2.Syntelog.isin(step7_excess[key])]

In [97]:
outfile = "/Users/fekeann/Documents/Lab/RNAseq/Syntelogs/final_selections/publication_output/"
outfile8_1 = outfile+"syntelogs_atl_only.csv"
outfile8_2 = outfile+"syntelogs_3species.csv"

In [98]:
#export just the syntelog and geneID columns
outsets = [step8_1, step8_2]
outfiles = [outfile8_1, outfile8_2]
for i, df in enumerate(outsets):
    df.drop(["species", "chromosome"], axis=1, inplace=True)
    df.to_csv(outfiles[i], index=False)